# Setup Notebook

In [1]:
%run analysis__setup.ipynb

The data has N = 128 rows and N = 56 columns.
Dropping N = 64 rows, keeping N = 64


## Prepare Data


In [10]:
# Filter out a single configuration to keep data manageable
df_agg = df_agg_full[
  (df_agg_full["sett_eval_fairness_grouping"] == "nationality-all") #&
  #(df_agg_full["sett_eval_exclude_subgroups"] == "keep-in-eval") &
  #(df_agg_full["sett_eval_on_subset"] == "full")
][
  # Remove eval columns
  cols_non_eval + cols_performance + cols_fairness
].reset_index(drop = True)

df_agg.shape

(32, 48)

# Calculate Variable Importance

## Use a Lasso Regression to estimate Importance of Settings

In [11]:
X = df_agg[cols_non_eval] # empty 
y = df_agg[main_fairness_metric]

### Main Effects of Settings Only (i.e. no interations)

In [12]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LassoCV

# Do preprocessing in a separate pipeline from model fitting
# (for eli5 to work)
preprocessor = make_pipeline(
    OneHotEncoder(),
).fit(X)
X_processed = preprocessor.transform(X)

lasso_reg = LassoCV(cv=5, random_state=0).fit(X_processed, y)

# Check whether it's predictive at all
lasso_reg.score(X_processed, y)


0.7907406068943852

Weights of importance:

In [13]:
import eli5
eli5.show_weights(lasso_reg, top=-1, feature_names = preprocessor.get_feature_names_out())


Weight?,Feature
+0.122,<BIAS>
+0.069,sett_model_elasticnet
+0.033,sett_model_logreg
+0.000,sett_cutoff_quantile_0.25
-0.008,sett_model_gbm
-0.019,sett_model_rf
-0.060,sett_cutoff_quantile_0.1


### Including Interactions

In [14]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LassoCV

# Do preprocessing w/o a pipeline for eli5 to work
preprocessor = make_pipeline(
    OneHotEncoder(),
    PolynomialFeatures(degree=2),
).fit(X)
X_processed = preprocessor.transform(X)

lasso_reg = LassoCV(cv=5, random_state=0).fit(X_processed, y)

# Check whether it's predictive at all
lasso_reg.score(X_processed, y)


0.9993964835958836

Weights of importance:

In [15]:
import eli5
eli5.show_weights(lasso_reg, top=-1, feature_names = preprocessor.get_feature_names_out())


Weight?,Feature
+0.111,<BIAS>
+0.098,sett_cutoff_quantile_0.25 sett_model_elasticnet
+0.035,sett_cutoff_quantile_0.25 sett_model_logreg
+0.015,sett_model_elasticnet
+0.009,sett_exclude_features_none sett_model_logreg
+0.007,sett_exclude_subgroups_keep-all sett_model_logreg
+0.007,sett_cutoff_quantile_0.25^2
+0.006,sett_exclude_features_sex sett_model_logreg
+0.004,sett_exclude_features_nationality-sex sett_model_rf
+0.002,sett_exclude_features_sex sett_model_gbm


## Use a functinoal ANOVA (fANOVA) to Analyze Setting Importance

Based on the following paper:

Hutter, F., Hoos, H., & Leyton-Brown, K. (2014). An Efficient Approach for Assessing Hyperparameter Importance. Proceedings of the 31st International Conference on Machine Learning, 754–762. https://proceedings.mlr.press/v32/hutter14.html


In [20]:
from fairness_multiverse.analysis import MultiverseFanova

m_fanova = MultiverseFanova(features = df_agg[cols_non_eval], outcome = df_agg[main_fairness_metric])


In [21]:
m_fanova.quantify_individual_importance()

AttributeError: 'SwigPyObject' object has no attribute 'mean'

In [ ]:
m_fanova.quantify_importance(save_to = ANALYSIS_OUTPUT_DIR / "fanova_importance_interactions-overall.csv")

In [ ]:
best_p_margs = m_fanova.fanova.get_most_important_pairwise_marginals(n=5)
print(best_p_margs)

In [ ]:
from fanova import visualizer

vis = visualizer.Visualizer(m_fanova.fanova, m_fanova.configuration_space, directory = str(ANALYSIS_OUTPUT_DIR))

In [ ]:
vis.plot_marginal(0)

In [ ]:
vis.plot_pairwise_marginal(['sett_exclude_features', 'sett_exclude_subgroups'])

## Quantify Importance with Partial Data

In [ ]:
from tqdm import tqdm

PARTIAL_FANOVA_DIR = ANALYSIS_OUTPUT_DIR / "partial_fanova" / "overall"
PARTIAL_FANOVA_DIR.mkdir(parents=True, exist_ok=True)

N_ITERATIONS = 10

In [ ]:
from fairness_multiverse.analysis import MultiverseFanova
import joblib

def quantify_importance_for_fraction(fraction: float, base_directory = PARTIAL_FANOVA_DIR):
    # Get random subset of the data
    df = df_agg.sample(frac = fraction).reset_index(drop = True)
    data_hash = joblib.hash(df)

    # Create directory for this fraction
    directory = base_directory / f"fraction-{fraction}"
    directory.mkdir(exist_ok = True)

    # Run FANOVA on subset
    partial_fanova = MultiverseFanova(features = df[cols_non_eval], outcome = df[main_fairness_metric])
    partial_fanova.quantify_importance(save_to = directory / f"partial-fanova_importance_interactions-majmin-{fraction}-{data_hash}.csv")

In [ ]:
for i in tqdm(range(N_ITERATIONS)):
    quantify_importance_for_fraction(0.01)

In [ ]:
for i in tqdm(range(N_ITERATIONS)):
    quantify_importance_for_fraction(0.05)

In [ ]:
for i in tqdm(range(N_ITERATIONS)):
    quantify_importance_for_fraction(0.1)

In [ ]:
for i in tqdm(range(N_ITERATIONS)):
    quantify_importance_for_fraction(0.2)